# Setting Up

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install torch

In [ ]:
! pip install -q transformers[sentencepiece] fastai ohmeow-blurr nbdev

In [ ]:
! pip install -q onnx==1.10.0 onnxruntime-gpu onnxruntime_tools

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install -q onnxruntime==1.12.0


In [ ]:
!pip show onnx




Name: onnx
Version: 1.14.0
Summary: Open Neural Network Exchange
Home-page: https://github.com/onnx/onnx
Author: ONNX
Author-email: onnx-technical-discuss@lists.lfaidata.foundation
License: Apache License v2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, protobuf, typing-extensions
Required-by: onnxruntime-tools


In [ ]:
!pip install --pre --no-cache-dir --no-deps --no-color --only-binary :all: onnx


In [ ]:
!pip install -q onnxruntime-gpu onnxruntime-tools


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoConfig
from fastai.text.all import *
from blurr.text.data.all import *
from blurr.text.modeling.all import *

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")


In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/ml-classifier

/content/drive/MyDrive/ml-classifier


# Data

In [ ]:
df = pd.read_csv("book_details_with_50colors.csv")
df.head()

,title,description,top_genres,colors
0,Awakening Your Money Chakras Secrets to Attracting Wealth and Success,"Welcome to our enlightening guide on ""Awakening Your Money Chakras: Secrets to Attracting Wealth and Success."" If you've ever wondered how some people effortlessly attract abundance into their lives while others struggle, this book has the answers you seek!\n\nIn this empowering read, we delve into the fascinating world of chakras, the energy centers within us that influence our financial well-being. From the Root Chakra to the Crown Chakra, we explore how each one impacts our money mindset and ability to manifest prosperity.\n\nLearn how to identify blockages and imbalances that may have ...","["" 'Nonfiction'"", "" 'Business & Economics'"", "" 'New Age'"", "" 'Meditation'"", "" 'Personal finance / money management'"", "" 'Religion & Spirituality'"", "" 'Body"", ' mind', "" & spirit'"", "" 'Inspiration & personal growth'""]","['firebrick', 'mediumseagreen', 'darkviolet', 'magenta', 'mediumblue', 'limegreen', 'seagreen', 'tomato', 'blue', 'silver']"
1,Emotions A to Z,"""The Empathy Weave: Embracing Emotional Intelligence for Meaningful Connections"" is a captivating and transformative exploration of emotional intelligence and empathy—a tapestry of understanding and compassion that weaves hearts together with profound human connections. In this enlightening book, readers embark on a journey of self-discovery and growth, unlocking the power of emotional intelligence to enhance personal relationships, professional success, and overall well-being.\n\nDrawing upon the rich tapestry of emotional intelligence, this book delves deep into the intricacies of self-a...","["" 'Nonfiction'"", "" 'Inspiration'"", "" 'General self-help'"", "" 'Self-improvement'"", "" 'Anxieties & Phobias'"", ""['Nonfiction'"", "" 'Anxieties & Phobias']""]","['firebrick', 'teal', 'mediumslateblue', 'darkturquoise', 'salmon', 'blue', 'crimson']"
2,The Art of Raise Capital for a New Venture,"""The Art of Raise Capital for Your New Venture"" is a practical and insightful guide that equips entrepreneurs with the knowledge and strategies needed to secure capital for their startup ventures. Authored with the understanding of the challenges faced by startup founders, this book serves as a valuable resource for aspiring entrepreneurs and seasoned founders alike.\n\nAt the core of successful fundraising lies the art of visionary storytelling. The book emphasizes the importance of articulating a powerful vision that resonates with potential investors. Through effective communication, en...","["" 'Business & Economics'"", "" 'Nonfiction'"", "" 'Starting up'"", "" 'Budgeting'"", ""['Nonfiction'"", "" 'Budgeting']""]","['mediumseagreen', 'firebrick', 'cyan', 'teal', 'blue', 'slategrey']"
3,райдуга навколо сонця,"Життя заможного бізнесмена Марка можна побачити лише у казках: квартири, дорогі автомобілі, красива дівчина. Проте одна подія змінює Марка: врятувавши дівчинку з полум'я та сам ледь не загинувши, він полишає все та вирішує піти у паломництво до Єрусалиму у пошуках відповідей на свої запитання. Шлях далекий та складний, сповнений пригод та небезпек. Але Марк впевнений, що там, біля Гробу Господнього, на нього чекає відповідь на найбільшу таємницю.","["" 'Adventure'"", "" 'Fiction'"", "" 'General'"", "" 'Travel'"", ""['Fiction'"", "" 'Travel']""]","['salmon', 'slategrey', 'gold', 'darkviolet', 'cadetblue', 'indianred']"
4,Sibiloi: The Genesis of Humanity?,"ONE Mysterious belt in an antique shop in London is a compass to secret caves of the Amalek tribe in Sibiloi, Northern Kenya.\n\nFOUR Researchers want to unravel the truth: biologist Dr Martha Watkins, anthropologist Jim Trevor, and archeologists Dr Paul Brando from California, US, and Professor Simiyu from Kenya.\n\nTWO Extremist organisations want the truth buried forever: a religious sect and a terror organisation.\n\nDeep in the caves of Sibiloi in Turkana, Kenya, is the answer that has evaded resear

In [ ]:
df = df.dropna().reset_index(drop=True)
df.shape

(21204, 4)

In [ ]:
colors_list = df.colors.to_list()
color_count = {}
for colors in colors_list:
  colors_list = eval(colors)
  for color in colors_list:
    if color in color_count.keys():
      color_count[color] += 1
    else:
      color_count[color] = 1
print(f"Number of colors for different Genres: {len(color_count)}")
print(color_count)

threshold = int(len(df) * 0.01)
rare_colors = [key for key, value in color_count.items() if value < threshold]
len(rare_colors)

Number of colors for different Genres: 58
{'firebrick': 20592, 'mediumseagreen': 3674, 'darkviolet': 3306, 'magenta': 1827, 'mediumblue': 1402, 'limegreen': 4004, 'seagreen': 3068, 'tomato': 1973, 'blue': 14904, 'silver': 1257, 'teal': 1985, 'mediumslateblue': 2084, 'darkturquoise': 2408, 'salmon': 1668, 'crimson': 3066, 'cyan': 2643, 'slategrey': 4392, 'gold': 5111, 'cadetblue': 4252, 'indianred': 1177, 'saddlebrown': 1962, 'orangered': 3508, 'yellow': 5030, 'darkgoldenrod': 3043, 'orange': 915, 'mediumspringgreen': 1599, 'olive': 1505, 'darkorange': 2608, 'lavender': 1873, 'rosybrown': 1853, 'brown': 3034, 'pink': 2404, 'orchid': 1831, 'navy': 2251, 'indigo': 1944, 'royalblue': 2471, 'mediumvioletred': 2116, 'red': 3054, 'steelblue': 2618, 'deeppink': 2620, 'darkkhaki': 3061, 'palevioletred': 1290, 'chocolate': 1534, 'green': 2283, 'slateblue': 1418, 'coral': 1250, 'mediumorchid': 1480, 'turquoise': 1874, 'purple': 3674, 'lime': 2911, 'darkcyan': 1576, 'peru': 2826, 'sienna': 1134, '

0

In [ ]:
colors_list = df.colors.to_list()
revised_color_list = []
indices_to_drop = []

for idx, colors in enumerate(colors_list):
  color_list = eval(colors)
  revised_colors = []

  for color in color_list:
    if color not in rare_colors:
      revised_colors.append(color)

  if len(revised_colors) == 0:
    indices_to_drop.append(idx)
  else:
    revised_color_list.append(revised_colors)

df = df.drop(indices_to_drop).reset_index(drop=True)
df['revised_colors'] = revised_color_list
df.shape

(21204, 5)

In [ ]:
revised_colors_list = df.revised_colors.to_list()
revised_color_count = {}
for colors in revised_colors_list:
  color_list = colors
  for color in color_list:
    if color in revised_color_count.keys():
      revised_color_count[color] += 1
    else:
      revised_color_count[color] = 1
print(f"Number of colors for different Genres: {len(revised_color_count)}")

Number of colors for different Genres: 58


In [ ]:
encode_color_types = { key: idx for idx, (key, value) in enumerate(revised_color_count.items())}
with open("color_types_encoded.json", "w") as fp:
  json.dump(encode_color_types, fp)

# We need this because for multilabel classification all genres have possibility to be present in the predictions
categorical_color_list = []
revised_colors_list = df.revised_colors.to_list()

for revised_colors in revised_colors_list:
  categorical_list = [0] * len(encode_color_types)
  for color in revised_colors:
    color_type_index = encode_color_types[color]
    categorical_list[color_type_index] = 1
  categorical_color_list.append(categorical_list)

df['color_cat_list'] = categorical_color_list
df.shape

(21204, 6)

In [ ]:
labels = list(encode_color_types.keys())
len(labels), labels[:5]

(58, ['firebrick', 'mediumseagreen', 'darkviolet', 'magenta', 'mediumblue'])

# Data Split

In [ ]:
splitter = RandomSplitter(valid_pct=0.1, seed=42)
train_ids, valid_ids = splitter(df)
len(train_ids), len(valid_ids)

(19084, 2120)

In [ ]:
valid_df = df.loc[valid_ids]
valid_df.head()

,title,description,top_genres,colors,revised_colors,color_cat_list
17286,भारतीय स्मारकों का संरक्षण,"भारत का इतिहास समृद्ध और संरक्षण के योग्य है। भारत में मंदिरों, मस्जिदों, मकबरों, प्राचीन शहर के खंडहरों, महलों और अन्य स्मारकों की संख्या चौंका देने वाली है। आज भी पुरातत्वविदों को नए-नए स्थानों पर महान स्मारकों के अवशेष मिल रहे हैं।\n\nदुख की बात है कि विभिन्न कारकों जैसे विकास, क्षरण आदि ने हमारी प्राचीन विरासत को नुकसान पहुंचाया है और अभी भी नुकसान पहुंचा रहे हैं। शहरों का अनियोजित विस्तार, नई सड़कों और शॉपिंग मॉल और रिहायशी इलाकों का निर्माण, अप्रतिबंधित पर्यटन, ये सब अक्सर हमारी पुरातत्व विरासत को नुकसान पहुंचाते हैं।\n\nइस पुस्तक में हम पाठकों को प्राचीन इमारतों और स्मारकों के संरक्...","["" 'Art"", ' Architecture', "" Photography'"", "" 'Architecture'"", "" 'Historic preservation'"", "" 'Nonfiction'"", "" 'General'"", "" 'Restoration techniques'"", ""['Nonfiction'"", "" 'Restoration techniques']""]","['gold', 'darkkhaki', 'deeppink', 'seagreen', 'purple', 'firebrick', 'gold', 'cyan', 'blue', 'salmon']","[gold, darkkhaki, deeppink, seagreen, purple, firebrick, gold, cyan, blue, salmon]","[1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
815,Sous Vide Cookbook: 575 Best Sous Vide Recipes of All Time,"The only sous vide cookbook you will ever need!\n\nThe best part of Sous Vide, you ask?\n\nRegardless of the level of your experience, you will be able to prepare the ""Perfect"" meal every single time.\n\nNow, this might sound a little bit difficult to believe at first, but it's actually true.\n\nBefore exploring the more than 500 amazing recipes in this book, I would ask you to walk through this in-depth introductory chapter that will introduce you to the basics of Sous Vide cooking and help you master the art.\n\nThis book is targeted toward amateurs and new chefs alike, so everything is ...","["" 'Cooking"", ' Food', ' Wine', "" Spirits'"", "" 'Nonfiction'"", "" 'Vegetarian & Vegan'"", "" 'Specific Ingredients / Meat'"", "" 'Specific Ingredients / Poultry'"", "" 'Methods / Professional'"", "" 'Specific ingredients / seafood'""]","['maroon', 'steelblue', 'limegreen', 'darkorange', 'firebrick', 'crimson', 'mediumseagreen', 'peru', 'purple', 'palevioletred']","[maroon, steelblue, limegreen, darkorange, firebrick, crimson, mediumseagreen, peru, purple, palevioletred]","[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0]"
14900,Vapor,"After Cindy's husband admits a painful secret, she discovers a satisfying way to make her presence known. Although it is mostly out of her control, she uses it to its full advantage, almost as good as having X-Ray vision!","["" 'Fiction'"", "" 'Horror'"", "" 'Ghost'"", "" 'Fantasy'"", "" 'Short stories'"", ""['Fiction'"", "" 'Short stories']""]","['slategrey', 'rosybrown', 'crimson', 'navy', 'mediumseagreen', 'cadetblue', 'mediumorchid']","[slategrey, rosybrown, crimson, navy, mediumseagreen, cadetblue, mediumorchid]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
17257,Ultimate Guide to Understanding Soft Skills for Interior Designers,"Interior designers like any other professionals require a lot of qualities to become successful in their careers. Soft skills like communication, time management, critical thinking, etc. play a vital role in their professional progress. This is a perfect ebook with comprehensive guidelines as to how to be great in soft skills, which are also known as emotional intelligence. This ebook is important for interior design students and professionals alike as it can help enhance professional success. The handbook covers a variety of topics that can help you improve your professional personality.\...

We will be using `valid_df` for all inference testing

# Fastai & Blurr Inference

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/ml-classifier

/content/drive/MyDrive/ml-classifier


In [ ]:
model_path = "models/rainbow-genre-cover-classifier-stage-01.pkl"
learner_inf = load_learner(model_path)

In [ ]:
learner_inf.blurr_predict("random placeholder")

[{'labels': ['slategrey', 'cadetblue'],
  'scores': [0.9016855359077454, 0.5734331011772156],
  'class_indices': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'class_labels': ['firebrick', 'mediumseagreen', 'darkviolet', 'magenta', 'mediumblue', 'limegreen', 'seagreen', 'tomato', 'blue', 'silver', 'teal', 'mediumslateblue', 'darkturquoise', 'salmon', 'crimson', 'cyan', 'slategrey', 'gold', 'cadetblue', 'indianred', 'saddlebrown', 'orangered', 'yellow', 'darkgoldenrod', 'orange', 'mediumspringgreen', 'olive', 'darkorange', 'lavender', 'rosybrown', 'brown', 'pink', 'orchid', 'navy', 'indigo', 'royalblue', 'mediumvioletred', 'red', 'steelblue', 'deeppink', 'darkkhaki', 'palevioletred', 'choc

In [ ]:
learner_inf.blurr_predict("random placeholder")[0]['labels']

['slategrey', 'cadetblue']

## Evaluation

In [ ]:
from sklearn import metrics

def metric_measures(test_df, preds):

  targets = [np.asarray(target) for target in test_df['color_cat_list'].to_list()]
  outputs = [np.asarray(pred) for pred in preds]


  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')

  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

  return

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=len(valid_df)):
  desc = row['description']
  labels = learner_inf.blurr_predict(desc)[0]['labels']
  pred_colors = [0] * len(encode_color_types)
  for label in labels:
    pred_colors[encode_color_types[label]] = 1
  preds.append(pred_colors)

preds[0][:20]

  0%|          | 0/2120 [00:00<?, ?it/s]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.7058906228033178
F1 Score (Macro) = 0.6126106404508097


# Convert to ONNX

### ONNX
ONNX (Open Neural Network Exchange) is a open standard for representing machine learning models.
- It allows developers to move models between different frameworks (such as PyTorch, TensorFlow, and Caffe2) without losing performance or accuracy.
ONNX makes it easier to build and run AI models on a variety of hardware, including GPUs, CPUs, and custom accelerators.
- This standard helps eliminate the need for multiple conversion tools and provides a unified representation of the model across different tools and frameworks.
- ONNX is being developed by a collaboration of companies including Microsoft, Facebook, Amazon, and IBM, among others, making it a well-supported and widely-adopted standard.
- Converting to ONNX runtime often makes model faster.

In [ ]:
model_path = "models/rainbow-genre-cover-classifier-stage-01.pkl"
learner_inf = load_learner(model_path)

In [ ]:
classifier = learner_inf.model.hf_model.eval()

torch.onnx.export(
    classifier,
    torch.LongTensor([[0] * 512]),
    'models/rainbow-genre-cover-classifier.onnx',
    input_names=['input_ids'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_len'},
        'output': {0: 'batch_size'}
    }
)

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model_path = 'models/rainbow-genre-cover-classifier.onnx'
quantized_onnx_model_path = 'models/rainbow-genre-cover-classifier-quantized.onnx'

quantize_dynamic(
    onnx_model_path,
    quantized_onnx_model_path,
    weight_type=QuantType.QUInt8,
)

Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/roberta/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant

# ONNX Inference

## Normal ONNX

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/ml-classifier

/content/drive/MyDrive/ml-classifier


In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_color_types.keys())

# Set the providers parameter to use CPUExecutionProvider as fallback
providers = ['CPUExecutionProvider']
inf_session = rt.InferenceSession('models/rainbow-genre-cover-classifier.onnx', providers=providers)

input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name


In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_colors = [0] * len(encode_color_types)
  for label in labels:
    pred_colors[encode_color_types[label]] = 1
  preds.append(pred_colors)

  0%|          | 0/2120 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds)

F1 Score (Micro) = 0.6973647125140925
F1 Score (Macro) = 0.6004194903584532


## Quantized ONNX

In [ ]:
import onnxruntime as rt
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

class_labels = list(encode_color_types.keys())

# Set the providers parameter to use CPUExecutionProvider as fallback
providers = ['CPUExecutionProvider']
inf_session = rt.InferenceSession('models/rainbow-genre-cover-classifier-quantized.onnx', providers=providers)

input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows(), total=valid_df.shape[0]):
  desc = row['description']
  input_ids = tokenizer(desc)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]
  probs = torch.FloatTensor(probs)

  masks = torch.sigmoid(probs) >= 0.5
  labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  pred_colors = [0] * len(encode_color_types)
  for label in labels:
    pred_colors[encode_color_types[label]] = 1
  preds.append(pred_colors)

  0%|          | 0/2120 [00:00<?, ?it/s]

In [ ]:
metric_measures(valid_df, preds) #

F1 Score (Micro) = 0.6962127884446011
F1 Score (Macro) = 0.5984456234321722
